In [2]:
import numpy as np
from sklearn.linear_model import Lasso
import utils
from CoRT_builder import CoRT
import importlib
import matplotlib.pyplot as plt
importlib.reload(utils)

<module 'utils' from 'D:\\Data Science Project\\CorT SI\\utils.py'>

In [14]:
import numpy as np
import heapq
import utils # Ensure the file above is named utils.py
from sklearn.linear_model import Lasso
from CoRT_builder import CoRT # Your existing builder
import parametric
import time
import importlib
importlib.reload(utils)

n_target = 50
n_source = 10
p = 100
K = 5
Ka = 3
h = 30
lamda = 0.03
s_vector = [0,0,0,0,0,0,0,0,0,0]
T = 5
s = len(s_vector)
iteration = 500
CoRT = CoRT(alpha=lamda)
z_min = zk = -20
z_max = 20

list_zk = [zk]

target_data, source_data = CoRT.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
similar_source_index = CoRT.find_similar_source(n_target, K, target_data, source_data, T=T, verbose=False)

X_combined, y_combined = CoRT.prepare_CoRT_data(similar_source_index, source_data, target_data)

model = Lasso(alpha=lamda, fit_intercept=False, random_state=42)
model.fit(X_combined, y_combined.ravel())
beta_hat_target = model.coef_[-p:]

active_indices = np.array([i for i, b in enumerate(beta_hat_target) if b != 0])

if len(active_indices) == 0:
  print(f"Iteration {iter}: Lasso selected no features. Skipping.")

j = np.random.choice(len(active_indices))

X_target = target_data["X"]
y_target = target_data["y"]
X_active, X_inactive = utils.get_active_X(beta_hat_target, X_target)
etaj, etajTy = utils.construct_test_statistic(y_target, j, X_active)


# ... (Setup Code: Generate Data, Center Data, Run CoRT, Setup Statistic) ...
# Assuming standard variables: n_target, p, target_data, etc. exist from previous steps

# 1. Setup Static Parameters
folds = utils.split_target(T, X_target, y_target, n_target)
Sigma = np.eye(n_target)
eta_sigma_eta = (etaj.T @ Sigma @ etaj).item()
b_global = (Sigma @ etaj) / eta_sigma_eta
a_global = y_target - b_global * etajTy

# 2. Initialize Trackers (The Heavy Lift - Done Once)
print("Initializing Path Trackers...")
model_configs = utils.precompute_matrices_for_tracker(folds, source_data, a_global, b_global, K, T)

trackers = []
pq = [] # Priority Queue

z_start = -20
z_end = 20

for i, conf in enumerate(model_configs):
    # Initialize Tracker
    tracker = utils.LassoPathTracker(conf["X"], conf["a"], conf["b"], lamda, z_start)
    trackers.append(tracker)
    
    # Get First Event
    next_z, evt, idx = tracker.get_next_knot()
    
    # Push to Heap: (z_time, model_index, event_type, var_index)
    heapq.heappush(pq, (next_z, i, evt, idx))

print(f"Initialized {len(trackers)} trackers.")

# 3. Event Loop
current_z = z_start
valid_intervals = []

# Prepare CoRT Tracker (Optional: if we assume active set of CoRT changes)
# For simplicity, let's just check CoRT validity at the midpoint of intervals
# To be perfectly exact, you'd add a tracker for CoRT too.

print("Starting Event-Driven Scan...")

while current_z < z_end:
    t1 = time.time()
    # 1. Get Next Event
    if not pq:
        next_event_z = z_end
    else:
        next_event_z, model_idx, evt_type, var_idx = pq[0] # Peek
        
    # Cap at z_end
    if next_event_z > z_end:
        next_event_z = z_end
        
    # 2. Check Interval Validity: [current_z, next_event_z]
    # In this interval, ALL Training Active Sets are CONSTANT.
    # We only need to check:
    #   a. Validation Voting Stability (Quadratic Roots)
    #   b. CoRT Stability (Linear)
    
    if next_event_z > current_z + 1e-6:
        # Check mid-point to represent the interval
        z_mid = (current_z + next_event_z) / 2
        
        # --- Check Validation ---
        L_val, R_val = -np.inf, np.inf
        
        # We need to construct the quadratics. 
        # Since trackers are frozen, we grab their u, v.
        # This part requires mapping t, k back to trackers.
        # Mapping: we know the order in 'trackers' list matches 'model_configs'
        
        # Re-organize u, v by (t, k)
        uv_map = {}
        for idx, conf in enumerate(model_configs):
            uv_map[(conf["t"], conf["k"])] = (trackers[idx].u, trackers[idx].v)
            
        # Run Voting Check (Algebraic)
        for t in range(T):
            u_b, v_b = uv_map[(t, -1)]
            conf_b = model_configs[t * (K+1)] # Baseline config usually first in group
            
            C2_b, C1_b, C0_b = utils.get_loss_coefs(conf_b["val"]["a"], conf_b["val"]["b"], u_b, v_b, conf_b["val"]["X"])
            
            for k in range(K):
                u_a, v_a = uv_map[(t, k)]
                C2_a, C1_a, C0_a = utils.get_loss_coefs(conf_b["val"]["a"], conf_b["val"]["b"], u_a, v_a, conf_b["val"]["X"])
                
                L_v, R_v = utils.solve_quadratic_interval(C2_a - C2_b, C1_a - C1_b, C0_a - C0_b, etajTy)
                L_val = max(L_val, L_v)
                R_val = min(R_val, R_v)
        
        # --- Check CoRT ---
        # Run standard check at z_mid
        L_CoRT, R_CoRT = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, z_mid)
        
        # Combine
        # Note: L_train/R_train are implicitly satisfied by the event loop structure itself!
        # The interval [current_z, next_event_z] IS the stability region for training.
        
        L_final = max(L_val, L_CoRT, current_z)
        R_final = min(R_val, R_CoRT, next_event_z)
        
        if L_final < R_final:
             # Check if observed statistic is in this valid range
             if L_final <= etajTy <= R_final:
                 print(f"** HIT ** Interval [{L_final:.4f}, {R_final:.4f}]")
                 valid_intervals.append((L_final, R_final))
    
    # 3. Update State
    current_z = next_event_z
    
    if current_z >= z_end:
        break
        
    # Process the Event (Pop and Update)
    z_evt, mod_id, evt, v_idx = heapq.heappop(pq)
    
    # Update the specific tracker
    trackers[mod_id].update_state(z_evt, evt, v_idx)
    
    # Calculate NEW next knot for this tracker
    new_z, new_evt, new_idx = trackers[mod_id].get_next_knot()
    
    # Push back
    heapq.heappush(pq, (new_z, mod_id, new_evt, new_idx))
    #t2 = time.time() - t1
    #print(t2)

print("Done.")

Initializing Path Trackers...
Initialized 30 trackers.
Starting Event-Driven Scan...
** HIT ** Interval [0.0706, 0.0737]
Done.
